Create a stack from of subtypes maps from all task and trial

In [1]:
import seaborn as sns
import os
import numpy as np
from scipy import stats, integrate
import scipy.io as sio
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import seaborn as sns; sns.set(color_codes=True)
import matplotlib.pyplot as plt

In [2]:
# load good stuff
%matplotlib inline
%load_ext rpy2.ipython
%load_ext oct2py.ipython
sns.set(color_codes=True)

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
#  Set path
path_root = '/home/yassinebha/Drive/HCP/subtypes_scores/26-10-2016/'
list_subtype_folder = ['subtype_5_spm_EMOTION_23-May-2017','subtype_5_spm_GAMBLING_19-Jun-2017','subtype_5_spm_LANGUAGE_18-Jun-2017',
                      'subtype_5_spm_MOTOR_19-Jun-2017','subtype_5_spm_RELATIONAL_18-Jun-2017','subtype_5_spm_SOCIAL_18-Jun-2017',
                       'subtype_5_spm_WM_18-Jun-2017']
list_subtype = ['sub1','sub2','sub3','sub4','sub5']

In [22]:
for cx,folder in enumerate(list_subtype_folder):
    path_network =  os.path.join(path_root,folder,'networks/')
    list_trial  = [f for f in os.listdir(path_network)]
    for ix,trial_name in enumerate(list_trial) :
        mat_file = os.path.join(path_network,trial_name,'subtype_{}.mat'.format(trial_name))
        %octave_push mat_file
        %octave mat_load = load(mat_file);
        %octave_pull mat_load
        subtype = mat_load.sub.map.T
        subtype_name = ['{}_{}'.format(subt,trial_name) for subt in list_subtype ]
        if ix == 0 and cx == 0:
            subtype_final = subtype
            subtype_name_final = subtype_name
        else:
            subtype_final = [np.concatenate([subtype_final[x], subtype[x]]) for x in range(np.shape(subtype)[0])]
            subtype_name_final = np.concatenate([subtype_name_final, subtype_name]) 

In [23]:
# make dataframe 
all_stack_df = pd.DataFrame(subtype_final,columns=subtype_name_final)
all_stack_df.describe()

,sub1_fear,sub2_fear,sub3_fear,sub4_fear,sub5_fear,sub1_win,sub2_win,sub3_win,sub4_win,sub5_win,...,sub1_tools,sub2_tools,sub3_tools,sub4_tools,sub5_tools,sub1_0bk,sub2_0bk,sub3_0bk,sub4_0bk,sub5_0bk
count,59900.000000,59900.000000,59900.000000,59900.000000,59900.000000,59900.000000,59900.000000,59900.000000,59900.000000,59900.000000,...,59900.000000,59900.000000,59900.000000,59900.000000,5.990000e+04,59900.000000,59900.000000,59900.000000,59900.000000,59900.000000
mean,-0.000201,0.000033,0.000143,-0.000142,0.000110,0.001284,-0.001223,-0.000457,-0.001691,0.001723,...,0.000255,-0.000048,-0.000056,0.000068,1.945685e-05,0.000060,-0.000138,-0.000052,-0.000059,0.000161
std,0.000394,0.000265,0.000433,0.000224,0.000227,0.001719,0.001757,0.001820,0.001530,0.001547,...,0.001201,0.000252,0.000121,0.000165,1.232127e-04,0.000177,0.000408,0.000148,0.000171,0.000476
min,-0.002871,-0.001305,-0.001174,-0.001607,-0.000593,-0.006037,-0.010259,-0.008793,-0.011401,-0.003734,...,-0.003325,-0.001542,-0.000777,-0.000659,-3.786224e-04,-0.000555,-0.002576,-0.001230,-0.001156,-0.001010
25%,-0.000266,-0.000090,-0.000089,-0.000206,-0.000017,0.000116,-0.001974,-0.001562,-0.002449,0.000591,...,-0.000362,-0.000090,-0.000094,-0.000035,-4.722838e-05,-0.000052,-0.000178,-0.000090,-0.000112,-0.000083
50%,-0.000095,0.000034,0.000034,-0.000080,0.000067,0.000942,-0.000723,-0.000539,-0.001324,0.001544,...,0.000007,0.000004,-0.000025,0.000035,9.939853e-07,0.000025,-0.000020,-0.000024,-0.000017,0.000028
75%,0.000004,0.000173,0.000245,-0.000011,0.000200,0.002229,-0.000048,0.000436,-0.000612,0.002708,...,0.000504,0.000080,0.000016,0.000149,5.390414e-05,0.000145,0.000070,0.000023,0.000037,0.000227
max,0.000457,0.000992,0.002656,0.000440,0.001794,0.009994,0.004227,0.008378,0.001986,0.008156,...,0.007636,0.000686,0.000252,0.000853,7.108363e-04,0.001018,0.000769,0.001090,0.000377,0.003013


In [24]:
# Make matrix out of the remaining guys except subject ID column
stack_mat = all_stack_df.as_matrix()
# Normalize these guys by column
stack_norm = (stack_mat - np.mean(stack_mat, 0))/np.std(stack_mat, 0)

In [25]:
# Save normalized pheno to be used in octave for bootsrap analysis
ind = all_stack_df.index
stack_norm_df = pd.DataFrame(stack_norm, columns=all_stack_df.columns.get_values(),index=ind)

In [26]:
stack_norm_df.describe()

,sub1_fear,sub2_fear,sub3_fear,sub4_fear,sub5_fear,sub1_win,sub2_win,sub3_win,sub4_win,sub5_win,...,sub1_tools,sub2_tools,sub3_tools,sub4_tools,sub5_tools,sub1_0bk,sub2_0bk,sub3_0bk,sub4_0bk,sub5_0bk
count,5.990000e+04,5.990000e+04,5.990000e+04,5.990000e+04,5.990000e+04,5.990000e+04,5.990000e+04,5.990000e+04,5.990000e+04,5.990000e+04,...,5.990000e+04,5.990000e+04,5.990000e+04,5.990000e+04,5.990000e+04,5.990000e+04,5.990000e+04,5.990000e+04,5.990000e+04,5.990000e+04
mean,-3.416299e-17,-2.277533e-17,1.062849e-16,2.941813e-17,-1.897944e-17,3.036710e-17,6.642804e-18,-1.897944e-17,-5.314243e-17,1.518355e-16,...,4.555065e-17,-5.504037e-17,3.416299e-17,8.350953e-17,1.708149e-17,4.934654e-17,-6.453009e-17,1.803047e-17,-2.277533e-17,-4.365271e-17
std,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,...,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00
min,-6.779992e+00,-5.043368e+00,-3.045255e+00,-6.535944e+00,-3.098077e+00,-4.257783e+00,-5.144508e+00,-4.579613e+00,-6.347996e+00,-3.527377e+00,...,-2.981206e+00,-5.920280e+00,-5.967904e+00,-4.402386e+00,-3.230857e+00,-3.472583e+00,-5.975433e+00,-7.968762e+00,-6.424800e+00,-2.458868e+00
25%,-1.661483e-01,-4.648026e-01,-5.362138e-01,-2.819993e-01,-5.618848e-01,-6.794724e-01,-4.275403e-01,-6.073963e-01,-4.958961e-01,-7.315548e-01,...,-5.144020e-01,-1.690098e-01,-3.159222e-01,-6.232201e-01,-5.412249e-01,-6.341439e-01,-9.672307e-02,-2.576956e-01,-3.125546e-01,-5.112028e-01
50%,2.687589e-01,9.361297e-04,-2.523591e-01,2.783182e-01,-1.928515e-01,-1.989040e-01,2.847685e-01,-4.546472e-02,2.395005e-01,-1.155269e-01,...,-2.068144e-01,2.052158e-01,2.500335e-01,-2.011043e-01,-1.498467e-01,-1.964711e-01,2.889118e-01,1.918372e-01,2.458096e-01,-2.794237e-01
75%,5.209267e-01,5.258668e-01,2.366550e-01,5.856178e-01,3.970142e-01,5.496503e-01,6.691011e-01,4.903049e-01,7.052044e-01,6.368307e-01,...,2.070567e-01,5.065809e-01,5.887840e-01,4.911605e-01,2.795782e-01,4.781140e-01,5.098467e-01,5.038369e-01,5.644734e-01,1.391901e-01
max,1.671758e+00,3.614897e+00,5.810362e+00,2.600095e+00,7.423557e+00,5.065581e+00,3.102656e+00,4.853627e+00,2.403550e+00,4.158089e+00,...,6.145438e+00,2.908665e+00,2.544137e+00,4.750975e+00,5.611314e+00,5.408944e+00,2.223652e+00,7.725171e+00,2.554118e+00,5.991879e+00


In [29]:
# save final stack
stack_norm_df.to_csv(os.path.join(path_root,'pheno/all_subtype_stack.csv'),index=False)

In [27]:
os.path.join(path_root,'pheno/all_subtype_stack.csv')

'/home/yassinebha/Drive/HCP/subtypes_scores/26-10-2016/pheno/all_subtype_stack.csv'

In [ ]:
'/home/yassinebha/Drive/HCP/subtypes_scores/26-10-2016/pheno/all_subtype_stack.csv'
